In [1]:
## 사전 실행 코드
import polars as pl
df_boxoffice = pl.read_csv("D:/book/polars/data/movie/KOBIS_기간별박스오피스.csv", skip_rows = 4, try_parse_dates=True)
boxoffice_cols = df_boxoffice.columns
boxoffice_cols[4] = '매출액점유율'
df_boxoffice.columns = [i.replace(' ', '') for i in boxoffice_cols]
df_boxoffice = df_boxoffice.with_columns(
    pl.col('매출액', '누적매출액').str.replace_all(',', '').cast(pl.Int64),
    pl.col('관객수', '누적관객수', '스크린수', '상영횟수').str.replace_all(',', '').cast(pl.Int64)    
)
df_boxoffice = df_boxoffice.with_columns(
    pl.col('장르').str.split(','), 
    pl.col('배우').str.split(','), 
)
df_boxoffice = (df_boxoffice.filter(
    pl.col('순위').is_not_null()).
                filter(~(pl.col('장르').list.contains('성인물(에로)'))).
                filter(pl.col('매출액') != 0).
                filter(~((pl.col('관객수') <= 1) & (pl.col('누적관객수') <= 1) & 
                         (pl.col('스크린수') <= 1) & (pl.col('상영횟수') <= 1)))
                )
import altair as alt
import plotly.express as px

# 8장 폴라스로 영화 데이터 생생하게 그리기

## 8.1	폴라스 시각화의 특징 알아보기

## 8.2	폴라스로 손쉽게 데이터 시각화하기

## 8.3	영화 데이터를 시각적으로 풍부하게 표현하기

## 8.3.5	선 그래프로 비교하는 〈파묘〉와 〈범죄도시 4〉 관객 추이

In [2]:
df_exhuma = (pl.read_csv("./KOBIS_일자별_통계정보_파묘.csv", skip_rows = 3, try_parse_dates = True)
            .with_columns(pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수')
                          .str.replace_all(',', '').cast(pl.Int64),
                          pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율')
                          .str.replace_all('%', '').cast(pl.Float64))
            .with_columns(pl.col('매출액').diff().alias('매출액변화'), pl.col('관객수').diff().alias('관객수변화'))
            .with_columns(pl.lit('파묘').alias('영화명')))

In [3]:
df_roundup = (pl.read_csv("./KOBIS_일자별_통계정보_범죄도시.csv", skip_rows = 3, try_parse_dates = True)
             .with_columns(pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수')
                           .str.replace_all(',', '').cast(pl.Int64),
                           pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율')
                           .str.replace_all('%', '').cast(pl.Float64))
             .with_columns(pl.col('매출액').diff().alias('매출액변화'), pl.col('관객수').diff().alias('관객수변화'))
             .with_columns(pl.lit('범죄도시4').alias('영화명')))

In [4]:
df_merge = pl.concat([df_exhuma, df_roundup], how = "vertical")
df_merge

날짜,스크린수,스크린점유율,상영횟수,상영점유율,좌석수,좌석점유율,좌석판매율,매출액,매출액증감(전일대비),관객수,관객수증감(전일대비),누적매출액,누적관객수,순위,매출액변화,관객수변화,영화명
date,i64,f64,i64,f64,i64,f64,f64,i64,str,i64,str,i64,i64,i64,i64,i64,str
2024-02-20,20,0.3,20,0.1,5173,0.2,100.0,49104000,"""49104000 ( 100.0% )""",5173,"""5173 ( 100.0% )""",49104000,5173,7,null,null,"""파묘"""
2024-02-21,5,0.1,5,0.0,838,0.0,100.0,8602000,"""-40502000 ( -82.5% )""",838,"""-4335 ( -83.8% )""",57706000,6011,20,-40502000,-4335,"""파묘"""
2024-02-22,1944,29.9,7535,46.2,1155876,51.2,28.6,3091398151,"""3082796151 ( 35838.1% )""",330118,"""329280 ( 39293.6% )""",3149104151,336129,1,3082796151,329280,"""파묘"""
2024-02-23,2086,30.4,8253,47.6,1273398,53.1,29.4,3656005238,"""564607087 ( 18.3% )""",374492,"""44374 ( 13.4% )""",6805109389,710621,1,564607087,44374,"""파묘"""
2024-02-24,2355,32.8,9473,51.8,1439628,57.2,53.6,7570365687,"""3914360449 ( 107.1% )""",770974,"""396482 ( 105.9% )""",14375475076,1481595,1,3914360449,396482,"""파묘"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-08-17,2,0.0,2,0.0,298,0.0,0.7,11000,"""-535000 ( -98.0% )""",2,"""-88 ( -97.8% )""",110038399734,11501617,114,-535000,-88,"""범죄도시4"""
2024-08-18,2,0.0,2,0.0,413,0.0,0.5,11000,"""0 ( 0.0% )""",2,"""0 ( 0.0% )""",110038410734,11501619,118,0,0,"""범죄도시4"""
2024-08-19,1,0.0,1,0.0,35,0.0,2.9,5000,"""-6000 ( -54.5% )""",1,"""-1 ( -50.0% )""",110038415734,11501620,102,-6000,-1,"""범죄도시4"""


In [5]:
df_merge.plot.line(x = '날짜', y = '관객수', color = '영화명').properties(width = 800, height = 350)

alt.Chart(...)